# Evaluation-V1

### Load Questions

In [81]:
import os
import pandas as pd

folder = "../data/Dataset/"
language = "en"
file = "spinach.tsv"
file_path = os.path.join(folder, language, file)

df_questions = pd.read_csv(file_path, sep="\t", encoding="utf-8")
df_questions["type"] = df_questions["type"].apply(lambda x: str(x) if not pd.isna(x) else "0")
df_questions

,usparql,Q1,Q2,Q3,Q4,type
0,no_usparql_13,Which television shows were created by John Cl...,What television programs did John Cleese create?,Which television shows were created by John Cl...,Which television shows were created by John Cl...,0
1,no_usparql_17,Give me all current communist countries.,What countries have a communist government?,Give me all communist countries in Asia.,Give me all communist countries outside of Asia.,0
2,no_usparql_26,Which politicians were married to a German per...,Which politicians had a spouse who was German?,Which politicians were married to a German woman?,Which politicians were married to a German man...,0
3,no_usparql_29,Give me all soccer clubs in Spain that play in...,Can you list every soccer club located in Spai...,Please tell me all the Spanish football clubs ...,Please tell me all the Spanish football clubs ...,0
4,no_usparql_32,Which telecommunications organizations are loc...,Which telecommunications organizations are bas...,Which telecommunications organizations are loc...,Which telecommunications organizations are loc...,0
...,...,...,...,...,...,...
145,no_usparql_3730,Which monarchs of the United Kingdom were marr...,Which British monarchs had spouses of German o...,Which monarchs of the United Kingdom were marr...,Which monarchs of the United Kingdom were marr...,0
146,no_usparql_3743,Give me all writers that won the Nobel Prize i...,List all authors who have received the Nobel P...,Give me all Asian writers that won the Nobel P...,Give me all writers from outside Asia that wo...,0
147,no_usparql_3744,Give me English actors starring in Lovesick.,Which English actors appear in Lovesick?,Give me English male actors starring in Lovesick.,Give me English female or non-binary actors st...,0
148,no_usparql_3802,Which rivers flow into the North Sea?,Which rivers have their mouth on the North Sea?,Which rivers flow through Britain into the Nor...,Which rivers flow into the North Sea but not t...,0


### Load Answers

In [1]:
import os

folder = "../data/answers/"
actions = ["fixing", "classification","wikidata"]
tasks = ['equal', 'sup-sub', "minus"]
datasets = ['spinach',"qawiki",'synthetic']
llms = ['gpt-4.1-2025-04-14', 'gpt-4.1-mini-2025-04-14', 'gpt-4.1-nano-2025-04-14', 'gpt-4o',"o3"]
questions = ["Q1", "Q2", "Q3", "Q4"]
# List to store the full paths of JSON files

json_files = []
# Recursively walk through the directory
for root, dirs, files in os.walk(folder):
    for file in files:
        if file.endswith(".json"):
            full_path = os.path.join(root, file)
            json_files.append(full_path)

print(f"JSON files found:{len(json_files)}")


JSON files found:1185


In [237]:
import json
df_answers = pd.DataFrame()
df_answers = pd.DataFrame(columns=["Q_ID", "Q_serie", "action", "task", "dataset", "llm"])

for file in json_files:
    elements = file.replace("_", "/")
    elements = elements.replace(".json", "")
    elements = elements.split("/")
    question = None
    action = "zero-shot"
    task = None
    dataset = None
    llm = None
    for q in questions:
        if q in elements:
            question = q
            break
    for a in actions:
        if a in elements:
            action = a
            break
    for t in tasks:
        if t in elements:
            task = t
            break
    for d in datasets:
        if d in elements:
            dataset = d
            break
    for l in llms:
        if l in elements:
            llm = l
            break
    if question and action and task and dataset and llm:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        df = pd.DataFrame([
                {"Q_ID": key, "Answer": value}
                for key, value in data.items()
            ])
        df["Q_serie"] = question
        df["action"] = action
        df["task"] = task
        df["dataset"] = dataset
        df["llm"] = llm
        df_answers = pd.concat([df_answers, df], ignore_index=True)
        
df_answers["Question"] = df_answers.apply(
    lambda x: df_questions.at[int(x["Q_ID"]), x["Q_serie"]] if int(x["Q_ID"]) in df_questions.index else None,
    axis=1
)

df_answers  

,Q_ID,Q_serie,action,task,dataset,llm,Answer,Question
0,45,Q1,classification,sup-sub,spinach,gpt-4o,[],Give me all spacecraft that flew to Mars.
1,46,Q1,classification,sup-sub,spinach,gpt-4o,[],Give me all taikonauts.
2,47,Q1,classification,sup-sub,spinach,gpt-4o,[],Which countries have more than ten volcanoes i...
3,48,Q1,classification,sup-sub,spinach,gpt-4o,"[Neil Armstrong, Buzz Aldrin, Michael Collins]",Who were the crew members on the Apollo 11 mis...
4,49,Q1,classification,sup-sub,spinach,gpt-4o,[],Give me all B-sides of the Ramones.
...,...,...,...,...,...,...,...,...
14343,145,Q2,zero-shot,equal,spinach,gpt-4.1-nano-2025-04-14,"[George the Third, Queen Victoria]",Which British monarchs had spouses of German o...
14344,146,Q2,zero-shot,equal,spinach,gpt-4.1-nano-2025-04-14,"[Gabriel Garcia Marquez, Toni Morrison, Albert...",List all authors who have received the Nobel P...
14345,147,Q2,zero-shot,equal,spinach,gpt-4.1-nano-2025-04-14,[Johnny Flynn],Which English actors appear in Lovesick?
14346,148,Q2,zero-shot,equal,spinach,gpt-4.1-nano-2025-04-14,"[River Thames, River Rhine, River Meuse, River...",Which rivers have their mouth on the North Sea?


In [238]:
df_answers = df_answers.copy()
# Define the group keys including Q_serie
group_keys = ["Q_ID", "Q_serie", "action", "dataset", "llm"]

# Assign unique case IDs for repeated rows with same key
df_answers["Answer_serie"] = (
    df_answers.groupby(group_keys)
    .cumcount()
    .apply(lambda x: x + 1)
)

df_answers.drop_duplicates(
    subset=["Q_ID", "Q_serie", "action", "task", "dataset", "llm"],
    inplace=True
)

df_answers.reset_index(drop=True, inplace=True)
df_answers

,Q_ID,Q_serie,action,task,dataset,llm,Answer,Question,Answer_serie
0,45,Q1,classification,sup-sub,spinach,gpt-4o,[],Give me all spacecraft that flew to Mars.,1
1,46,Q1,classification,sup-sub,spinach,gpt-4o,[],Give me all taikonauts.,1
2,47,Q1,classification,sup-sub,spinach,gpt-4o,[],Which countries have more than ten volcanoes i...,1
3,48,Q1,classification,sup-sub,spinach,gpt-4o,"[Neil Armstrong, Buzz Aldrin, Michael Collins]",Who were the crew members on the Apollo 11 mis...,1
4,49,Q1,classification,sup-sub,spinach,gpt-4o,[],Give me all B-sides of the Ramones.,1
...,...,...,...,...,...,...,...,...,...
14343,145,Q2,zero-shot,equal,spinach,gpt-4.1-nano-2025-04-14,"[George the Third, Queen Victoria]",Which British monarchs had spouses of German o...,1
14344,146,Q2,zero-shot,equal,spinach,gpt-4.1-nano-2025-04-14,"[Gabriel Garcia Marquez, Toni Morrison, Albert...",List all authors who have received the Nobel P...,1
14345,147,Q2,zero-shot,equal,spinach,gpt-4.1-nano-2025-04-14,[Johnny Flynn],Which English actors appear in Lovesick?,1
14346,148,Q2,zero-shot,equal,spinach,gpt-4.1-nano-2025-04-14,"[River Thames, River Rhine, River Meuse, River...",Which rivers have their mouth on the North Sea?,1


In [282]:
len(df_answers["Q_ID"].unique())

150

### Analysis

In [ ]:
from utils import jaccard_similarity

def mark_logical_errors(df):
    df = df.copy()
    df["isEmpty"] = df["Answer"].apply(lambda x: 1 if isinstance(x, list) and len(x) == 0 else 0)

    # Initialize error flag columns
    for i in range(1, 6):
        df[f"consistency_{i}"] = 0


    df["jaccard_similarity(A1-A2)"] = -1
    df["jaccard_similarity(A1-A34)"] = -1
    

    # Group by the full combination
    group_keys = ["Q_ID", "action", "dataset", "llm"]
    grouped = df_answers[df_answers["Answer_serie"]==1].groupby(group_keys)
    
    for keys, group in grouped:
        # Ensure we have all four Q-series
        if set(group["Q_serie"]) >= {"Q1", "Q2", "Q3", "Q4"}:

            A1 = set(group[group["Q_serie"] == "Q1"]["Answer"].values[0]) if not group[group["Q_serie"] == "Q1"].empty else set()
            A2 = set(group[group["Q_serie"] == "Q2"]["Answer"].values[0]) if not group[group["Q_serie"] == "Q2"].empty else set()
            A3 = set(group[group["Q_serie"] == "Q3"]["Answer"].values[0]) if not group[group["Q_serie"] == "Q3"].empty else set()
            A4 = set(group[group["Q_serie"] == "Q4"]["Answer"].values[0]) if not group[group["Q_serie"] == "Q4"].empty else set()

            # print(f"Processing group: {keys} with answers: {A1}, {A2}, {A3}, {A4}")
            # Evaluate logical conditions
            check = {
                "consistency_1": A1 == A2,
                "consistency_2": A1 == A3.union(A4),
                "consistency_3": A3.issubset(A1),
                "consistency_4": A4.issubset(A1),
                "consistency_5": A3.isdisjoint(A4)
            }

            similarity = { 
                 "jaccard_similarity(A1-A2)": jaccard_similarity(A1, A2),
                "jaccard_similarity(A1-A34)": jaccard_similarity(A1, A3.union(A4))}
            

            for idx in group.index:
                for key, val in check.items():
                    if val:
                        df.at[idx, key] = 1
                for key, val in similarity.items():
                    df.at[idx, key] = float(f"{val:.3f}")
            
    return df


In [310]:
from utils import jaccard_similarity
import pandas as pd

def analysis(df):
    df = df.copy()
    summaries = []

    group_keys = ["Q_ID", "action", "dataset", "llm"]
    grouped = df[df["Answer_serie"] == 1].groupby(group_keys)

    for keys, group in grouped:
        if set(group["Q_serie"]) >= {"Q1", "Q2", "Q3", "Q4"}:
            # Answers from Answer_serie == 1
            A1 = set(group[group["Q_serie"] == "Q1"]["Answer"].values[0])
            A2 = set(group[group["Q_serie"] == "Q2"]["Answer"].values[0])
            A3 = set(group[group["Q_serie"] == "Q3"]["Answer"].values[0])
            A4 = set(group[group["Q_serie"] == "Q4"]["Answer"].values[0])

            # Questions
            q_map = {
                row["Q_serie"]: row["Question"]
                for _, row in group.iterrows()
                if row["Q_serie"] in {"Q1", "Q2", "Q3", "Q4"}
            }

            # A1_prime from Answer_serie == 2
            df_serie2 = df[
                (df["Answer_serie"] == 2) &
                (df["Q_serie"] == "Q1")
            ]
            for col, val in zip(group_keys, keys):
                df_serie2 = df_serie2[df_serie2[col] == val]
            A1_prime = set(df_serie2["Answer"].values[0]) if not df_serie2.empty else set()

            # A1_double_prime from Answer_serie == 3
            df_serie3 = df[
                (df["Answer_serie"] == 3) &
                (df["Q_serie"] == "Q1")
            ]
            for col, val in zip(group_keys, keys):
                df_serie3 = df_serie3[df_serie3[col] == val]
            A1_double_prime = set(df_serie3["Answer"].values[0]) if not df_serie3.empty else set()

            # Logical checks
            consistency = {
                "?A1=A2": int(A1 == A2),
                "?A1=A3+A4": int(A1 == A3.union(A4)),
                "?A1>A3": int(A3.issubset(A1)),
                "?A1>A4": int(A4.issubset(A1)),
                "?A3∅A4": int(A3.isdisjoint(A4)),
            }

            # Jaccard similarities
            similarities = {
                "J(A1-A2)": round(jaccard_similarity(A1, A2), 4),
                "J(A1-A34)": round(jaccard_similarity(A1, A3.union(A4)), 4),
                "J(A1-A1*)": round(jaccard_similarity(A1, A1_prime), 4),
                "J(A1-A1**)": round(jaccard_similarity(A1, A1_double_prime), 4)
            }

            # Count empty answers
            # is_empty_count = group["Answer"].apply(lambda x: isinstance(x, list) and len(x) == 0).sum()

            # Build summary
            summary_row = {
                "Q_ID": keys[0],
                "action": keys[1],
                "dataset": keys[2],
                "llm": keys[3],
                # "isEmpty_count": is_empty_count,
                **consistency,
                **similarities,
                "Q1": q_map.get("Q1", ""),
                "Q2": q_map.get("Q2", ""),
                "Q3": q_map.get("Q3", ""),
                "Q4": q_map.get("Q4", ""),
                "A1": list(A1),
                "A2": list(A2),
                "A3": list(A3),
                "A4": list(A4),
                "A1*": list(A1_prime),
                "A1**": list(A1_double_prime)
            }
            summaries.append(summary_row)

    return pd.DataFrame(summaries)


In [311]:
df_analysis = analysis(df_answers)
df_analysis

,Q_ID,action,dataset,llm,?A1=A2,?A1=A3+A4,?A1>A3,?A1>A4,?A3∅A4,J(A1-A2),...,Q1,Q2,Q3,Q4,A1,A2,A3,A4,A1*,A1**
0,0,classification,spinach,gpt-4.1-2025-04-14,0,0,1,0,1,0.3333,...,Which television shows were created by John Cl...,What television programs did John Cleese create?,Which television shows were created by John Cl...,Which television shows were created by John Cl...,"[Seven Deadly Sins, The Human Face, Whoops Apo...","[Seven Deadly Sins (1970 television series), F...","[Seven Deadly Sins, The Human Face, Whoops Apo...",[The Secret Policeman's Ball 2006],"[The Secret Policeman's Ball, Seven Deadly Sin...","[Seven Deadly Sins (1970 television series), F..."
1,0,classification,spinach,gpt-4.1-mini-2025-04-14,1,0,1,0,1,1.0000,...,Which television shows were created by John Cl...,What television programs did John Cleese create?,Which television shows were created by John Cl...,Which television shows were created by John Cl...,"[Monty Python's Flying Circus, A Fish Called W...","[Monty Python's Flying Circus, A Fish Called W...","[The Frost Report, Ripping Yarns, Fawlty Tower...",[A Fish Called Wanda (television adaptation)],"[The World of Wooster, Monty Python's Flying C...","[Monty Python's Flying Circus, A Fish Called W..."
2,0,classification,spinach,gpt-4.1-nano-2025-04-14,1,1,1,1,1,0.0000,...,Which television shows were created by John Cl...,What television programs did John Cleese create?,Which television shows were created by John Cl...,Which television shows were created by John Cl...,[],[],[],[],[],[]
3,0,fixing,spinach,gpt-4.1-2025-04-14,0,1,1,1,1,0.6667,...,Which television shows were created by John Cl...,What television programs did John Cleese create?,Which television shows were created by John Cl...,Which television shows were created by John Cl...,"[Hold the Sunset, Fawlty Towers, Monty Python'...","[Fawlty Towers, Monty Python's Flying Circus]","[Fawlty Towers, Monty Python's Flying Circus]",[Hold the Sunset],"[Hold the Sunset, Fawlty Towers, Monty Python'...","[Fawlty Towers, Monty Python's Flying Circus]"
4,0,fixing,spinach,gpt-4.1-mini-2025-04-14,0,0,1,0,1,0.6667,...,Which television shows were created by John Cl...,What television programs did John Cleese create?,Which television shows were created by John Cl...,Which television shows were created by John Cl...,"[Monty Python's Flying Circus, A Fish Called W...","[Monty Python's Flying Circus, A Fish Called W...","[The Frost Report, Ripping Yarns, Fawlty Tower...",[A Fish Called Wanda (television pilot)],"[Monty Python's Flying Circus, A Fish Called W...","[Monty Python's Flying Circus, A Fish Called W..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2501,99,zero-shot,spinach,gpt-4.1-2025-04-14,1,1,1,1,1,1.0000,...,Which states border Illinois?,What states share a border with Illinois?,Which states in the Great Lakes region border ...,Which states not in the Great Lakes region bor...,"[Indiana, Kentucky, Iowa, Wisconsin, Missouri]","[Indiana, Kentucky, Iowa, Wisconsin, Missouri]","[Indiana, Wisconsin]","[Kentucky, Iowa, Missouri]",[],[]
2502,99,zero-shot,spinach,gpt-4.1-mini-2025-04-14,1,0,0,1,0,1.0000,...,Which states border Illinois?,What states share a border with Illinois?,Which states in the Great Lakes region border ...,Which states not in the Great Lakes region bor...,"[Indiana, Kentucky, Iowa, Wisconsin, Missouri]","[Indiana, Kentucky, Iowa, Wisconsin, Missouri]","[Indiana, Wisconsin, Michigan]","[Indiana, Kentucky, Iowa, Wisconsin, Missouri]",[],[]
2503,99,zero-shot,spinach,gpt-4.1-nano-2025-04-14,0,0,1,0,0,0.6250,...,Which states border Illinois?,What states share a border with Illinois?,Which states in the Great Lakes region border ...,Which states not in the Great Lakes region bor...,"[Indiana, Kentucky, Iowa, Wisconsin, Missouri]","[Arkansas, Tennessee, Indiana, Kentucky, Iowa,...","[Indiana, Wisconsin, Iowa, Missouri]","[Kentucky, Missouri, Tennessee]",[],[]
2504,99,zero-shot,spinach,gpt-4o,1,1,1,1,1

In [313]:
def summary(df_analysis):
    """
    Compute group-wise statistics from the output of `analysis(df)`.
    Grouped by dataset, action, llm.
    Returns mean values for consistency ratios, similarities, p-values, and empty answer ratios.
    """
    group_cols = ["dataset", "action", "llm"]

    # Columns to average
    consistency_cols = ["?A1=A2", "?A1=A3+A4", "?A1>A3", "?A1>A4", "?A3∅A4"]
    jaccard_cols = ["J(A1-A2)", "J(A1-A34)", "J(A1-A1*)", "J(A1-A1**)"]
    pval_cols = [col for col in df_analysis.columns if col.startswith("p_value_")]
    metric_cols = consistency_cols + jaccard_cols + pval_cols

    # Add empty answer flags
    for a in ["A1", "A2", "A3", "A4"]:
        df_analysis[f"{a}_empty_ratio"] = df_analysis[a].apply(lambda x: int(isinstance(x, list) and len(x) == 0))

    empty_cols = [f"{a}_empty_ratio" for a in ["A1", "A2", "A3", "A4"]]

    # Filter out rows with invalid Jaccard scores
    df_valid = df_analysis.copy()
    for col in jaccard_cols:
        df_valid = df_valid[df_valid[col] != -1]

    # Compute means grouped by dataset, action, and llm
    df_summary = (
        df_valid
        .groupby(group_cols)[metric_cols + empty_cols]
        .mean()
        .reset_index()
        .round(4)
        .rename(columns={col: f"{col}" for col in metric_cols + empty_cols})
    )

    return df_summary


In [314]:
df_summery = summary(df_analysis)
df_summery

,dataset,action,llm,?A1=A2,?A1=A3+A4,?A1>A3,?A1>A4,?A3∅A4,J(A1-A2),J(A1-A34),J(A1-A1*),J(A1-A1**),A1_empty_ratio,A2_empty_ratio,A3_empty_ratio,A4_empty_ratio
0,spinach,classification,gpt-4.1-2025-04-14,0.4124,0.5155,0.9485,0.7113,0.8763,0.7204,0.8272,0.7598,0.7187,0.0103,0.0515,0.0515,0.1443
1,spinach,classification,gpt-4.1-mini-2025-04-14,0.3867,0.4467,0.9667,0.7200,0.8867,0.5929,0.6855,0.5664,0.5971,0.1000,0.1067,0.1400,0.2867
2,spinach,classification,gpt-4.1-nano-2025-04-14,0.8267,0.6667,0.8400,0.8867,0.9867,0.0589,0.0617,0.0000,0.0087,0.9000,0.8200,0.7400,0.8733
3,spinach,fixing,gpt-4.1-2025-04-14,0.6133,0.6333,0.9467,0.7067,0.9400,0.8176,0.8535,0.8417,0.8455,0.0400,0.0333,0.0733,0.1133
4,spinach,fixing,gpt-4.1-mini-2025-04-14,0.5067,0.5200,0.8867,0.7333,0.8467,0.7439,0.8074,0.7916,0.7759,0.0133,0.0200,0.0400,0.1267
5,spinach,fixing,gpt-4.1-nano-2025-04-14,0.2400,0.3333,0.6467,0.5667,0.5667,0.3337,0.4644,0.5406,0.3253,0.1733,0.1667,0.1400,0.1867
6,spinach,fixing,gpt-4o,0.5842,0.5644,0.9109,0.7129,0.8911,0.4864,0.5482,0.4952,0.4782,0.2871,0.3861,0.2970,0.3861
7,spinach,fixing,o3,0.1967,0.3443,0.9508,0.3934,0.8689,0.5327,0.7311,0.5594,0.5181,0.0492,0.0492,0.0328,0.0656
8,spinach,wikidata,gpt-4.1-2025-04-14,0.5000,0.2133,0.5667,0.4400,0.6467,0.7980,0.6200,0.0000,0.0000,0.0000,0.0000,0.0400,0.0933
9,spinach,wikidata,gpt-4.1-mini-2025-04-14,0.4400,0.1533,0.4867,0.4267,0.4867,0.7297,0.5670,0.0000,0.0000,0.0000,0.0000,0.0067,0.0467


# Evaluation -V2

In [13]:
import os
import json
import pandas as pd
from utils import jaccard_similarity
import datetime

def get_answer_set(df, q_serie, task):
    match = df[(df["Q_serie"] == q_serie) & (df["task"] == task)]
    if not match.empty:
        return set(match["Answer"].values[0])
    return set()

def load_question(file_path: str) -> pd.DataFrame:
    df = pd.read_csv(file_path, sep="\t", encoding="utf-8")
    # df["type"] = df["type"].apply(lambda x: str(x) if not pd.isna(x) else "0")
    return df



In [3]:
def load_all_questions(root_dir, datasets, languages):
    """
    Load and merge question files from multiple datasets and languages.

    Args:
        root_dir (str): Base directory containing the question files.
        datasets (list): List of dataset names.
        languages (list): List of language codes.
        load_questions_fn (Callable): Function to load a TSV file into a DataFrame.

    Returns:
        pd.DataFrame: Merged DataFrame with original index stored as 'q_index',
                      and columns 'dataset' and 'lang' added.
    """
    all_dfs = []

    for dataset in datasets:
        for lang in languages:
            question_path = os.path.join(root_dir, "data", "Dataset", lang, f"{dataset}.tsv")
            if not os.path.exists(question_path):
                print(f"File not found: {question_path}")
                continue

            df = load_question(question_path)
            df = df.copy()
            df["q_index"] = df.index
            df["dataset"] = dataset
            df["lang"] = lang

            all_dfs.append(df)

    return pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame()



In [4]:
def load_answers(folder: str, datasets, llms, actions, tasks, languages, questions) -> pd.DataFrame:
    df_answers = pd.DataFrame(columns=["Q_ID", "Q_serie", "action", "task", "dataset", "lang","llm"])

    json_files = [
        os.path.join(root, file)
        for root, _, files in os.walk(folder)
        for file in files if file.endswith(".json")
    ]

    print(f"JSON files found: {len(json_files)}")

    for file in json_files:
        if not file.split("/")[-1].startswith("Q"):
            continue
        elements = file.replace("_", "/").replace(".json", "").split("/")
        question = next((q for q in questions if q in elements), None)
        action = next((a for a in actions if a in elements), "zero-shot")
        task = next((t for t in tasks if t in elements), None)
        dataset = next((d for d in datasets if d in elements), None)
        lang = next((l for l in languages if l in elements), None)
        llm = next((l for l in llms if l in elements), None)

        if all([question, action, task, dataset, llm]):
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            df = pd.DataFrame([{"Q_ID": key, "Answer": value} for key, value in data.items()])
            df["Q_serie"] = question
            df["action"] = action
            df["task"] = task
            df["dataset"] = dataset
            df["llm"] = llm
            df["lang"] = lang
            df_answers = pd.concat([df_answers, df], ignore_index=True)

    return df_answers

In [10]:
def enrich_answers(df_answers, df_questions):
    df_answers["Question"] = df_answers.apply(
        lambda x: df_questions.loc[
            (df_questions["q_index"] == int(x["Q_ID"])) &
            (df_questions["dataset"] == x["dataset"])
        ][x["Q_serie"]].values[0]
        if not df_questions.loc[
            (df_questions["q_index"] == int(x["Q_ID"])) &
            (df_questions["dataset"] == x["dataset"]) 
        ].empty else None,
        axis=1
    )

    df_answers.drop_duplicates(
        subset=["Q_ID", "Q_serie", "action", "task", "dataset", "llm"],
        inplace=True
    )

    df_answers.reset_index(drop=True, inplace=True)
    return df_answers

In [63]:
def analysis(df):
    rows = []
    group_keys = ["Q_ID", "action", "dataset", "llm"]
    # grouped = df[df["Answer_serie"] == 1].groupby(group_keys)
    grouped = df.groupby(group_keys)

    for keys, group in grouped: 
        if set(group["Q_serie"]) >= {"Q1", "Q2", "Q3", "Q4"}:
            action = group["action"].values[0]
            if action in ["zero-shot", "wikidata"]:
                A1 = get_answer_set(group, "Q1", "equal")
                A2 = get_answer_set(group, "Q2", "equal")
                A3 = get_answer_set(group, "Q3", "sup-sub")
                A4 = get_answer_set(group, "Q4", "minus")

                A1_prime = None
                A1_double_prime = None

                similarities = {
                    "J(A1-A2)": round(jaccard_similarity(A1, A2), 4),
                    "J(A1-A34)": round(jaccard_similarity(A1, A3.union(A4)), 4),
                    "J(A1-A1*)": None,
                    "J(A1-A1**)": None,
                    "J(A1*-A1**)": None
                    }
                consistency = {
                    "?A1=A2": int(A1 == A2),
                    "?A1=A3+A4": int(A1 == A3.union(A4)),
                    "?A1>A3": int(A3.issubset(A1)),
                    "?A1>A4": int(A4.issubset(A1)),
                    "?A3∅A4": int(A3.isdisjoint(A4))
                    }
            elif action in ['classification','fixing']:
                # Usage
                A1_equal = get_answer_set(group, "Q1", "equal")
                A1_contain = get_answer_set(group, "Q1", "sup-sub")
                A1_minus = get_answer_set(group, "Q1", "minus")
                A2_equal = get_answer_set(group, "Q2", "equal")
                A3_contain = get_answer_set(group, "Q3", "sup-sub")
                A3_minus = get_answer_set(group, "Q3", "minus")
                A4_minus = get_answer_set(group, "Q4", "minus")
                similarities = {
                    "J(A1-A2)": round(jaccard_similarity(A1_equal, A2_equal), 4),
                    "J(A1-A34)": round(jaccard_similarity(A1_minus, A3_minus.union(A4_minus)), 4),
                    "J(A1-A1*)": round(jaccard_similarity(A1_equal, A1_contain), 4),
                    "J(A1-A1**)": round(jaccard_similarity(A1_equal, A1_minus), 4),
                    "J(A1*-A1**)": round(jaccard_similarity(A1_contain, A1_minus), 4)
                    }
                consistency = {
                    "?A1=A2": int(A1_equal == A2_equal),
                    "?A1=A3+A4": int(A1_minus == A3_minus.union(A4_minus)),
                    "?A1>A3": int(A3_contain.issubset(A1_contain)),
                    "?A1>A4": int(A4_minus.issubset(A1_minus)),
                    "?A3∅A4": int(A3_minus.isdisjoint(A4_minus))
                    }

                A1 = A1_equal
                A2 = A2_equal
                A3 = A3_contain
                A4 = A4_minus
                A1_prime = list(A1_contain)
                A1_double_prime = list(A1_minus)
                
            q_map = {
                row["Q_serie"]: row["Question"]
                for _, row in group.iterrows()
                if row["Q_serie"] in {"Q1", "Q2", "Q3", "Q4"}
            }

            row = {
                "Q_ID": keys[0], "action": keys[1], "dataset": keys[2], "llm": keys[3],
                **consistency, **similarities,
                "Q1": q_map.get("Q1", ""), "Q2": q_map.get("Q2", ""),
                "Q3": q_map.get("Q3", ""), "Q4": q_map.get("Q4", ""),
                "A1": list(A1), "A2": list(A2), "A3": list(A3), "A4": list(A4),
                "A1*": A1_prime, "A1**": A1_double_prime
            }
            rows.append(row)

    return pd.DataFrame(rows)

In [64]:
def summary(df_analysis):
    group_cols = ["dataset", "action", "llm"]
    consistency_cols = ["?A1=A2", "?A1=A3+A4", "?A1>A3", "?A1>A4", "?A3∅A4"]
    jaccard_cols = ["J(A1-A2)", "J(A1-A34)", "J(A1-A1*)", "J(A1-A1**)","J(A1*-A1**)"]
    pval_cols = [col for col in df_analysis.columns if col.startswith("p_value_")]
    metric_cols = consistency_cols + jaccard_cols + pval_cols

    for a in ["A1", "A2", "A3", "A4"]:
        df_analysis[f"{a}_empty_ratio"] = df_analysis[a].apply(lambda x: int(isinstance(x, list) and len(x) == 0))

    empty_cols = [f"{a}_empty_ratio" for a in ["A1", "A2", "A3", "A4"]]

    df_valid = df_analysis.copy()
    for col in jaccard_cols:
        df_valid = df_valid[df_valid[col] != -1]

    df_summary = (
        df_valid
        .groupby(group_cols)[metric_cols + empty_cols]
        .mean()
        .reset_index()
        .round(4)
    )

    return df_summary

In [24]:
root_dir = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
datasets=["spinach", "qawiki",'synthetic']
# llms = ['gpt-4.1-2025-04-14', 'gpt-4.1-mini-2025-04-14', 'gpt-4.1-nano-2025-04-14', 
#         'gpt-4o','o3','gpt-5-nano',"gpt-5-mini","gpt-5",
#         "gemini-2.0-flash","gemini-2.5-flash","gemini-2.5-pro",
#         "grok-3-mini","deepseek-chat","deepseek-reasoner","llama3.1:8b","llama3.3:70b"]
llms = ['gpt-4.1-2025-04-14']
actions = ["fixing", "classification", "wikidata"]
tasks = ['equal', 'sup-sub', "minus"]
languages = ['en']

df_questions = load_all_questions(root_dir, datasets, languages)


In [28]:
df_questions

,usparql,Q1,Q2,Q3,Q4,type,q_index,dataset,lang,q1,q2,Type,Unnamed: 7,0: fully containment
0,no_usparql_13,Which television shows were created by John Cl...,What television programs did John Cleese create?,Which television shows were created by John Cl...,Which television shows were created by John Cl...,NaN,0,spinach,en,NaN,NaN,NaN,NaN,NaN
1,no_usparql_17,Give me all current communist countries.,What countries have a communist government?,Give me all communist countries in Asia.,Give me all communist countries outside of Asia.,NaN,1,spinach,en,NaN,NaN,NaN,NaN,NaN
2,no_usparql_26,Which politicians were married to a German per...,Which politicians had a spouse who was German?,Which politicians were married to a German woman?,Which politicians were married to a German man...,NaN,2,spinach,en,NaN,NaN,NaN,NaN,NaN
3,no_usparql_29,Give me all soccer clubs in Spain that play in...,Can you list every soccer club located in Spai...,Please tell me all the Spanish football clubs ...,Please tell me all the Spanish football clubs ...,NaN,3,spinach,en,NaN,NaN,NaN,NaN,NaN
4,no_usparql_32,Which telecommunications organizations are loc...,Which telecommunications organizations are bas...,Which telecommunications organizations are loc...,Which telecommunications organizations are loc...,NaN,4,spinach,en,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,NaN,In which universities do Nobel Prize Laureates...,Which universities have Nobel laureates on the...,Which universities have Nobel laureates in Eco...,Which universities have Nobel laureates on the...,NaN,145,synthetic,en,NaN,NaN,NaN,NaN,NaN
446,NaN,With which countries does India have ongoing b...,Which countries currently have territorial dis...,With which countries that are nuclear powers d...,With which countries that are not nuclear powe...,NaN,146,synthetic,en,NaN,NaN,NaN,NaN,NaN
447,NaN,What countries on the Arabian Peninsula are co...,Which countries are in the Middle East and on ...,What countries on the Arabian Peninsula are pa...,What countries on the Arabian Peninsula are pa...,NaN,147,synthetic,en,NaN,NaN,NaN,NaN,NaN
448,NaN,What rivers over 500 kilometers long flow into...,Which rivers over 500 kms in length feed into ...,What rivers over 500 kilometers long flow into...,What rivers over 500 kilometers long flow into...,NaN,148,synthetic,en,NaN,NaN,NaN,NaN,NaN


In [44]:
df_answers = load_answers(
    folder=root_dir + "/data/answers/",
    datasets = datasets,
    llms=llms,
    actions=actions,
    tasks=tasks,
    languages=languages,
    questions=["Q1", "Q2", "Q3", "Q4"]
)


JSON files found: 1185


In [45]:
df_answers

,Q_ID,Q_serie,action,task,dataset,lang,llm,Answer
0,0,Q1,classification,sup-sub,spinach,None,gpt-4.1-2025-04-14,"[Fawlty Towers, Whoops Apocalypse, The Human F..."
1,1,Q1,classification,sup-sub,spinach,None,gpt-4.1-2025-04-14,"[People's Republic of China, Socialist Republi..."
2,2,Q1,classification,sup-sub,spinach,None,gpt-4.1-2025-04-14,"[Boris Johnson, Gerhard Schröder, Nicolas Sark..."
3,3,Q1,classification,sup-sub,spinach,None,gpt-4.1-2025-04-14,"[Real Club Deportivo Mallorca, Real Club Depor..."
4,4,Q1,classification,sup-sub,spinach,None,gpt-4.1-2025-04-14,"[Proximus Group, Telenet Group, Orange Belgium..."
...,...,...,...,...,...,...,...,...
10073,145,Q2,wikidata,equal,qawiki,None,gpt-4.1-2025-04-14,"[Norman H. Nie, C. Hadlai 'Tex' Hull, Dale H. ..."
10074,146,Q2,wikidata,equal,qawiki,None,gpt-4.1-2025-04-14,"[Fernando Haddad, Luiz Inácio Lula da Silva]"
10075,147,Q2,wikidata,equal,qawiki,None,gpt-4.1-2025-04-14,"[Goh Kun, Lee Hae-chan, Han Myeong-sook, Han D..."
10076,148,Q2,wikidata,equal,qawiki,None,gpt-4.1-2025-04-14,"[Marie Curie, Linus Pauling, John Bardeen, Fre..."


In [46]:
df_answers[
    (df_answers["dataset"] == "spinach") &
    (df_answers["action"] == "classification")
]["task"].value_counts()

task
minus      450
sup-sub    300
equal      194
Name: count, dtype: int64

In [ ]:
duplicated_rows = df_answers[df_answers.duplicated(
    subset=["Q_ID", "Q_serie", "action", "task", "dataset", "llm"],
    keep=False
)]

# Sort to group duplicates together
duplicated_rows_sorted = duplicated_rows.sort_values(
    by=["Q_ID", "Q_serie", "action", "task", "dataset", "llm"]
)

# Display the result
duplicated_rows_sorted
# the duplicated rows are due to multiple runs of wikidate with different order of name construction

,Q_ID,Q_serie,action,task,dataset,lang,llm,Answer
6794,0,Q1,wikidata,equal,spinach,None,gpt-4.1-2025-04-14,[Fawlty Towers]
6944,0,Q1,wikidata,equal,spinach,None,gpt-4.1-2025-04-14,"[Fawlty Towers, Fawlty Towers: Re-Opened]"
7394,0,Q2,wikidata,equal,spinach,None,gpt-4.1-2025-04-14,[Fawlty Towers]
7544,0,Q2,wikidata,equal,spinach,None,gpt-4.1-2025-04-14,"[Monty Python's Flying Circus, Fawlty Towers]"
6795,1,Q1,wikidata,equal,spinach,None,gpt-4.1-2025-04-14,"[People's Republic of China, Republic of Cuba,..."
...,...,...,...,...,...,...,...,...
7642,98,Q2,wikidata,equal,spinach,None,gpt-4.1-2025-04-14,"[Colorado, Idaho, Montana, New Mexico, Wyoming]"
6893,99,Q1,wikidata,equal,spinach,None,gpt-4.1-2025-04-14,"[Iowa, Missouri, Kentucky, Indiana, Wisconsin]"
7043,99,Q1,wikidata,equal,spinach,None,gpt-4.1-2025-04-14,"[Wisconsin, Iowa, Missouri, Kentucky, Indiana]"
7493,99,Q2,wikidata,equal,spinach,None,gpt-4.1-2025-04-14,"[Iowa, Missouri, Kentucky, Indiana, Wisconsin]"


In [57]:
df_answers = enrich_answers(df_answers, df_questions)

In [58]:
df_answers

,Q_ID,Q_serie,action,task,dataset,lang,llm,Answer,Question
0,0,Q1,classification,sup-sub,spinach,None,gpt-4.1-2025-04-14,"[Fawlty Towers, Whoops Apocalypse, The Human F...",Which television shows were created by John Cl...
1,1,Q1,classification,sup-sub,spinach,None,gpt-4.1-2025-04-14,"[People's Republic of China, Socialist Republi...",Give me all current communist countries.
2,2,Q1,classification,sup-sub,spinach,None,gpt-4.1-2025-04-14,"[Boris Johnson, Gerhard Schröder, Nicolas Sark...",Which politicians were married to a German per...
3,3,Q1,classification,sup-sub,spinach,None,gpt-4.1-2025-04-14,"[Real Club Deportivo Mallorca, Real Club Depor...",Give me all soccer clubs in Spain that play in...
4,4,Q1,classification,sup-sub,spinach,None,gpt-4.1-2025-04-14,"[Proximus Group, Telenet Group, Orange Belgium...",Which telecommunications organizations are loc...
...,...,...,...,...,...,...,...,...,...
9789,145,Q2,wikidata,equal,qawiki,None,gpt-4.1-2025-04-14,"[Norman H. Nie, C. Hadlai 'Tex' Hull, Dale H. ...",Who are SPSS' initial developers?
9790,146,Q2,wikidata,equal,qawiki,None,gpt-4.1-2025-04-14,"[Fernando Haddad, Luiz Inácio Lula da Silva]",Who ran against Bolsonaro for Brazilian Presid...
9791,147,Q2,wikidata,equal,qawiki,None,gpt-4.1-2025-04-14,"[Goh Kun, Lee Hae-chan, Han Myeong-sook, Han D...",Name all South Korean Prime Ministers from the...
9792,148,Q2,wikidata,equal,qawiki,None,gpt-4.1-2025-04-14,"[Marie Curie, Linus Pauling, John Bardeen, Fre...",Who has won multiple types of Nobel Prize?


In [65]:
df_analysis = analysis(df_answers)

In [66]:
df_analysis

,Q_ID,action,dataset,llm,?A1=A2,?A1=A3+A4,?A1>A3,?A1>A4,?A3∅A4,J(A1-A2),...,Q1,Q2,Q3,Q4,A1,A2,A3,A4,A1*,A1**
0,0,classification,qawiki,gpt-4.1-2025-04-14,1,1,1,1,1,1.0000,...,In which countries are tepuis found?,In which countries are tepuis located?,In which Spanish-speaking countries are tepuis...,In which countries that are not Spanish-speaki...,"[Venezuela, Brazil, Guyana]","[Venezuela, Brazil, Guyana]",[Venezuela],"[Brazil, Guyana]","[Venezuela, Brazil, Guyana]","[Venezuela, Brazil, Guyana]"
1,0,classification,spinach,gpt-4.1-2025-04-14,1,1,1,1,1,1.0000,...,Which television shows were created by John Cl...,What television programs did John Cleese create?,Which television shows were created by John Cl...,Which television shows were created by John Cl...,"[Seven Deadly Sins (1970 television series), F...","[Seven Deadly Sins (1970 television series), F...","[Seven Deadly Sins, Whoops Apocalypse, The Hum...",[The Secret Policeman's Ball 2006],"[Seven Deadly Sins, Whoops Apocalypse, The Hum...","[The Secret Policeman's Ball 2006, The Secret ..."
2,0,classification,synthetic,gpt-4.1-2025-04-14,1,1,1,1,1,1.0000,...,Name the films for which Quentin Tarantino was...,Which movies were directed by Quentin Tarantino?,Name the films directed by Quentin Tarantino s...,Name the films directed by Quentin Tarantino n...,"[Death Proof, Inglourious Basterds, Reservoir ...","[Death Proof, Inglourious Basterds, Reservoir ...",[Inglourious Basterds],"[Death Proof, Kill Bill Volume 2, Reservoir Do...","[Death Proof, Inglourious Basterds, Reservoir ...","[Death Proof, Kill Bill Volume 2, Inglourious ..."
3,0,fixing,qawiki,gpt-4.1-2025-04-14,1,1,1,1,1,1.0000,...,In which countries are tepuis found?,In which countries are tepuis located?,In which Spanish-speaking countries are tepuis...,In which countries that are not Spanish-speaki...,"[Venezuela, Brazil, Guyana]","[Venezuela, Brazil, Guyana]",[Venezuela],"[Brazil, Guyana]","[Venezuela, Brazil, Guyana]","[Venezuela, Brazil, Guyana]"
4,0,fixing,spinach,gpt-4.1-2025-04-14,1,1,1,1,1,1.0000,...,Which television shows were created by John Cl...,What television programs did John Cleese create?,Which television shows were created by John Cl...,Which television shows were created by John Cl...,"[Fawlty Towers, Monty Python's Flying Circus]","[Fawlty Towers, Monty Python's Flying Circus]","[Fawlty Towers, Monty Python's Flying Circus]",[Hold the Sunset],"[Fawlty Towers, Monty Python's Flying Circus, ...","[Fawlty Towers, Monty Python's Flying Circus, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1742,99,wikidata,spinach,gpt-4.1-2025-04-14,1,1,1,1,1,1.0000,...,Which states border Illinois?,What states share a border with Illinois?,Which states in the Great Lakes region border ...,Which states not in the Great Lakes region bor...,"[Indiana, Iowa, Missouri, Wisconsin, Kentucky]","[Indiana, Iowa, Missouri, Wisconsin, Kentucky]","[Indiana, Wisconsin]","[Missouri, Iowa, Kentucky]",None,None
1743,99,wikidata,synthetic,gpt-4.1-2025-04-14,1,1,1,1,0,1.0000,...,Which colors are on the flag of both Germany a...,What are the colors used for both the German a...,What primary colors are used for both the Germ...,What non-primary colors are used for both the ...,"[red, black, yellow]","[red, black, yellow]","[red, black, yellow]","[red, black, yellow]",None,None
1744,99,zero-shot,qawiki,gpt-4.1-2025-04-14,0,0,0,0,1,0.4808,...,What languages are pro-drop?,What languages drop pronouns?,Which pro-drop languages use Latin script?,Which pro-drop languages do not use Latin script?,"[Yoruba, Arabic, Aymara, Finnish, Romanian, Be...","[Yoruba, Indonesian, Finnish, Vietnamese, Mara...","[Romansh, French, Indonesian, Occitan, Bosnian...","[Arabic, Amharic, Urdu, Cantonese, Lao, Thai, ...",None,None
1745,99,zero-shot,spinach,gpt-4.1-2025-04-14,1,1,1,1,1,1.0000,...,Which states border Illinois?,What states share a border with Illinois?,Which states in the Great Lakes region border ...,

In [67]:
df_summary = summary(df_analysis)
df_summary

,dataset,action,llm,?A1=A2,?A1=A3+A4,?A1>A3,?A1>A4,?A3∅A4,J(A1-A2),J(A1-A34),J(A1-A1*),J(A1-A1**),J(A1*-A1**),A1_empty_ratio,A2_empty_ratio,A3_empty_ratio,A4_empty_ratio
0,qawiki,classification,gpt-4.1-2025-04-14,0.9067,0.8267,0.9333,0.9333,0.8133,0.9786,0.9630,0.6710,0.6530,0.6491,0.0000,0.0000,0.0000,0.0000
1,qawiki,fixing,gpt-4.1-2025-04-14,0.7600,0.7467,0.8667,0.9533,0.8933,0.9013,0.9098,0.7944,0.7890,0.7863,0.0067,0.0067,0.0200,0.0667
2,qawiki,wikidata,gpt-4.1-2025-04-14,0.2667,0.1267,0.3933,0.3467,0.6133,0.6657,0.5248,NaN,NaN,NaN,0.0000,0.0000,0.0200,0.0067
3,qawiki,zero-shot,gpt-4.1-2025-04-14,0.2800,0.0867,0.3600,0.3067,0.6200,0.5849,0.4417,NaN,NaN,NaN,0.0000,0.0000,0.0267,0.0267
4,spinach,classification,gpt-4.1-2025-04-14,0.9485,0.8969,0.9485,0.9691,0.9794,0.9249,0.9110,0.7187,0.6813,0.7598,0.0515,0.0515,0.0515,0.1443
5,spinach,fixing,gpt-4.1-2025-04-14,0.9067,0.7600,0.9467,0.9267,1.0000,0.9275,0.9078,0.8455,0.8494,0.8417,0.0333,0.0333,0.0733,0.1133
6,spinach,wikidata,gpt-4.1-2025-04-14,0.5000,0.2133,0.5667,0.4400,0.6467,0.7980,0.6200,NaN,NaN,NaN,0.0000,0.0000,0.0400,0.0933
7,spinach,zero-shot,gpt-4.1-2025-04-14,0.4733,0.2067,0.5200,0.4267,0.6333,0.7449,0.5839,NaN,NaN,NaN,0.0000,0.0000,0.0133,0.0533
8,synthetic,classification,gpt-4.1-2025-04-14,0.9000,0.8533,0.9667,0.9267,0.9400,0.9724,0.9629,0.8470,0.8308,0.8528,0.0000,0.0000,0.0000,0.0000
9,synthetic,fixing,gpt-4.1-2025-04-14,0.8867,0.8667,0.9533,0.9667,0.9733,0.9662,0.9711,0.9175,0.9131,0.9145,0.0000,0.0000,0.0133,0.0733


# TODO:

1. relation-classification analysis
 In the zero-shot, there are relation classification to see 
 - if the LLMs able to understand the relation between answers correctly. 
 - if incorrect, normally which ones misclassified to which ones? Probably confusion matrix here. 
    
2. internal inconsistency
 We observe that even we ask the exactly same questions multiple times, there are always with difference/inconsistency. 
 - how much these internal inconsistency they are? 
 - how to exclude them into the inconsistency between questions, say properly evaluate the inconsistency caused by questions.
 - the impact factor of these internal inconsistency to answers inconsistency? 

3. correct relation identification lead consistency answers? 
We have the relation classification in classification-and-question actions
- the inconsistency if LLMs identify relation correctly, as well as when they identify incorrectly;
- the impact factor of this relation identification to inconsistency? 

4. can we build the impact factors model to attribute the cause of inconsistency? 
 We can conclude some factors such as internal inconsistency, relation-identification capability (semantic understanding to text), ...

5. how can we mitigate the inconsistency? 
 we tried several actions, and let's do further detailed analysis to actions and consequencs. 
- wikidata: consistency improved, the trade-off is more "idk" answers.

6. an overall analysis include all datasets together. 


7. explain of how we get the data, and the properties of each datasets. 

8. empty ratio need be include "idk", rename is no answers or idk. 
